In [2]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return T4, P100, or A100


True
Tesla T4


In [3]:
pip install gcsfs --no-deps

In [4]:
pip install bigframes datasets

In [5]:
pip install bigframes datasets

In [6]:
from huggingface_hub import login
login(token="")

In [7]:
from datasets import load_dataset

In [8]:
import os
os.environ["HF_TOKEN"] = ""

In [9]:
dataset = load_dataset("theatticusproject/cuad")

Resolving data files:   0%|          | 0/203 [00:00<?, ?it/s]

In [10]:
print(dataset["train"].features)
print(dataset["train"][0])

{'text': Value(dtype='string', id=None)}
{'text': '================================================='}


In [11]:
pip install bitsandbytes

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "EleutherAI/gpt-j-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:  48%|####7     | 11.6G/24.2G [00:00<?, ?B/s]

Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.20.attn.bi

In [13]:
prompt_template = """
Generate an employee contract for {job_role} at {company_name}.
Include sections on:
1. Employment terms
2. Compensation
3. Responsibilities
4. Benefits
5. Termination conditions
6. Confidentiality agreement

The contract should follow standard legal terms.

"""

job_role = "Software Engineer"
company_name = "TechCorp"

prompt = prompt_template.format(job_role=job_role, company_name=company_name)


In [17]:
print(model)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear4bit(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear4bit(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )


In [18]:
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
input_ids = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")

print("Input IDs:", input_ids)

Input IDs: tensor([[  198,  8645,   378,   281,  6538,  2775,   329, 10442, 23164,   379,
          9634, 45680,    13,   220,   198,   818,  9152,  9004,   319,    25,
           198,    16,    13, 24656,  2846,   198,    17,    13, 39059,   198,
            18,    13, 20549,  7992,   198,    19,    13, 30582,   198,    20,
            13, 15527,   341,  3403,   198,    21,    13,  7326, 35599,   414,
          4381,   198,   198,   464,  2775,   815,  1061,  3210,  2742,  2846,
            13,   628]], device='cuda:0')


In [20]:
output = model.generate(
    input_ids,
    max_new_tokens=200,  # Generates 200 new tokens beyond input length
    pad_token_id=tokenizer.eos_token_id
)

contract_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(contract_text)



Generate an employee contract for Software Engineer at TechCorp. 
Include sections on:
1. Employment terms
2. Compensation
3. Responsibilities
4. Benefits
5. Termination conditions
6. Confidentiality agreement

The contract should follow standard legal terms.




In [1]:
from transformers import AutoModelForCausalLM



import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load GPT-J Model & Tokenizer
model_name = "EleutherAI/gpt-j-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", load_in_8bit=True)

# Define Prompt
prompt = """Generate an employee contract for a Software Engineer at TechCorp.
Include sections on:
1. Employment terms
2. Compensation
3. Responsibilities
4. Benefits
5. Termination conditions
6. Confidentiality agreement

The contract should follow standard legal terms.
"""

# Ensure the tokenizer has a pad token
tokenizer.pad_token = tokenizer.eos_token  # Fix padding issue

# Tokenize Input
input_ids = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")

# Generate Output
output = model.generate(
    input_ids,
    max_new_tokens=300,  # Ensure long enough output
    do_sample=True,  # Enables randomness for better results
    temperature=0.7,  # Controls creativity
    top_p=0.9,  # Nucleus sampling
    pad_token_id=tokenizer.eos_token_id
)

# Decode & Print
contract_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(contract_text)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transfor

Generate an employee contract for a Software Engineer at TechCorp.
Include sections on:
1. Employment terms
2. Compensation
3. Responsibilities
4. Benefits
5. Termination conditions
6. Confidentiality agreement

The contract should follow standard legal terms.
The following information is to be included in the contract:
1. Company name and address
2. Employee's full name
3. Employee's title
4. Employee's email address
5. Employee's phone number
6. Employee's address
7. Employee's Social Security Number
8. Employee's salary
9. Employee's hourly rate
10. Employee's start date
11. Employee's end date
12. Employee's email address
13. Employee's phone number
14. Employee's address
15. Employee's Social Security Number
16. Employee's title
17. Employee's responsibilities
18. Employee's benefits
19. Employee's termination date
20. Employee's start date
21. Employee's end date
22. Employee's responsibilities
23. Employee's benefits
24. Employee's termination date
25. Employee's start date
26. 